In [22]:
# !pip install qci-client
# !pip install sympy

In [1]:
import numpy as np
from scipy.sparse import dok_matrix
import os
from qci_client import QciClient
from sympy import IndexedBase, Matrix
from time import time
from helpers import extract_solution

In [3]:
qc = QciClient(api_token="M3YYWg1ZaEuWkKI2eCIxFaFMoKeYem5m", url="https://api.qci-next.com")

# Quadratic Knapsack

## Description

The general Knapsack problem is: given a capacity of items (the knapsack), select items to maximize the value of the knapsack. Quadratic Knapsack has the ability to represent when items selected together contribute additional value or detract from the overall value.

## Relevance

This arises with capital projects. For example:

1. Power grids and power generation are dependent upon each other for producing returns.
2. Retail market saturation detracts from total returns, considering capacity of each location to service customers.

## Example

Consider a market for power demand. Five different power grid projects and seven different power generation projects are available to choose from. 

Long term demand increase of 95GWd needs to be planned.

Five grid projects are up for review. Five power generation projects are up for review.

This problem is designed to select the right grid projects and the amount of power generation to plan from each plant.

In [4]:
grids = list(range(5))
plants = list(range(5))
association = [(0, 1), (0, 1, 2), (1, 2, 3), (2, 3, 4), (3, 4)]
return_pgw = [(4.405, 3.946), (3.896, 4.084, 4.227), (3.559, 3.700, 3.636), (4.208, 3.558, 4.168), (3.854, 3.513)]
# return_pgw = [(4.405, 3.946), (4.558, 0, 1.392), (3.259, 4.408, 1.636), (4.203, 3.984, 4.227), (0, 4.313)]
grid_detractors = [(0, 1), (1, 2), (2, 3), (3, 4)]
overlap_factor = [8.727, 9.724, 7.221, 9.009]
# overlap_factor = [0, 0, 0, 0]

In [9]:
n_qudits = 15
# set the returns (per day)
J = np.zeros((n_qudits, n_qudits))
for i, local_grids in enumerate(association):
    plant_idx = 10+i
    for j, grid_idx in enumerate(local_grids):
        # maximize
        J[grid_idx, plant_idx] = -return_pgw[i][j]
# set the grid overlap penalties
for i, (grid1, grid2) in enumerate(grid_detractors):
    J[grid1,grid2] = overlap_factor[i]

In [10]:

penalties_quad = np.zeros((n_qudits, n_qudits))
penalties_lin = np.zeros((n_qudits,1))
# a + b = 1
# (a + b - 1) ** 2
# a**2 + b**2 + 2*a*b - a - b + 1
# put the grid and auxiliary variables in :10
alpha = 12
grid_selector = np.arange(5)
aux_selector = np.arange(5, 10)
plant_selector = np.arange(10, 15)
penalties_quad[grid_selector, grid_selector] += alpha
penalties_quad[aux_selector, aux_selector] += alpha
penalties_quad[grid_selector, aux_selector] += 2*alpha
penalties_lin[grid_selector] -= alpha
penalties_lin[aux_selector] -= alpha

In [12]:

h = np.zeros((n_qudits,1))
# each grid costs 2k
h[:5] = 2
# each plant costs 2k/GW
h[10:] = 2
h += penalties_lin
# add penalties to J
J += penalties_quad
# make J symmetric
J += J.T
J /= 2
problem = np.hstack([h, J])
np.save("/workspace/leap/quadknapsacktest.npy", problem)


In [13]:
prob_vals = np.abs(problem)
prob_vals = prob_vals[prob_vals>0]
np.min(prob_vals), np.max(prob_vals)

(1.7565, 24.0)

In [14]:
t0 = time()
ham_file = {"file_type": "hamiltonian", "file_name": "quad_knapsack.json"}
data = []
problem = dok_matrix(problem)
for (i, j), value in problem.items():
    data.append({"i": int(i), "j": int(j), "val": float(value)})
ham_file["data"] = data
ham_file["num_variables"] = n_qudits
ham_file_id = qc.upload_file(ham_file)["file_id"]
print(f"File ID: {ham_file_id}")
print(f'total time {time()-t0}')

File ID: 64d2237ef25d8eafde6d948a
total time 0.36049747467041016


In [15]:
body = qc.build_job_body(job_type="sample-hamiltonian", hamiltonian_file_id=ham_file_id, 
                         job_params={"sampler_type": "eqc2", "n_samples": 2, "solution_type": "integer"},
                         job_name="quad_knapsack")

In [16]:
response = qc.process_job("sample-hamiltonian", body)

Job submitted job_id='64d2238af25d8eafde6d948d'-: 2023/08/08 11:14:19
RUNNING: 2023/08/08 11:14:20
COMPLETED: 2023/08/08 11:16:31


In [17]:
grid_selection, power_values = extract_solution(response["results"]["samples"][0], 5)

In [18]:
for grid in grid_selection:
    print(f"Power grid project {grid+1} selected for execution.")
for i, gw in enumerate(power_values):
    if gw > 0:
        print(f"Plant project {i+1} with {gw} GWd planned")

Power grid project [1 2 3] selected for execution.
Plant project 2 with 95.0 GWd planned


In [19]:
x = np.array(response["results"]["samples"][0])
x

array([ 5,  4,  6,  0,  0,  0,  0,  0,  0,  0,  0, 85,  0,  0,  0])

In [20]:
h.T@x+x.T@J@x

array([-2924.214])

In [21]:
response["results"]["energies"]

[-2924.214]